### IMBOR Ontologie Netwerk structuur

Dit notebook laat in een netwerk visualisatie zien hoe de IMBOR Objecttypen aan elkaar gerelateerd zijn

In [4]:
import json
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import sys
from rdflib import Graph,URIRef
from gastrodon import LocalEndpoint,one,QName
import pandas as pd
pd.set_option("display.width",100)
pd.set_option("display.max_colwidth",80)

We beginnen met het importeren van de libraries en het laden van de ontologie uit de turtle file...

In [2]:
g=Graph()
g.parse("data/imbor-0.5.2.ttl",format='turtle')
len(g)

50578

In [3]:
e=LocalEndpoint(g)

### 'Netwerk query'

In onderstaande query wordt vanaf een bepaald concept in de ontologie (in het voorbeeld hieronder 'Sloot') gezocht naar de onderliggende en de bovenliggende objecttypes. Het resultaat is een tabel met 'van' - 'naar' nodes en het type relatie.

In [32]:
q=e.select("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT distinct ?parent_name ?child_name ?type
WHERE
{
VALUES ?root_name { "Sloot"@nl-NL }
{
    ?root skos:prefLabel ?root_name .
    ?child rdfs:subClassOf+ ?root .
    ?child rdfs:subClassOf ?parent .
    ?child skos:prefLabel ?child_name .
    ?parent skos:prefLabel ?parent_name .
    BIND ("subclass" as ?type)
}
UNION 
{
    ?root skos:prefLabel ?root_name .
    ?sclass rdfs:subClassOf ?root .
    ?sclass rdfs:subClassOf+ ?child .
    ?child rdfs:subClassOf ?parent .
    ?child skos:prefLabel ?child_name .
    ?parent skos:prefLabel ?parent_name .
    BIND ("superclass" as ?type)
   FILTER (?parent_name != 'physical object'@en)
}
}ORDER BY ?type ?parent_name
    """)
q

,parent_name,child_name,type
0,Sloot,Boezemspoorsloot,subclass
1,Sloot,Vaart,subclass
2,Sloot,Spoorsloot,subclass
3,Sloot,Te verlanden sloot,subclass
4,Sloot,Scheisloot,subclass
5,Sloot,Poldersloot,subclass
6,Sloot,Dijksloot,subclass
7,Sloot,Tocht,subclass
8,Sloot,Bermsloot,subclass
9,Sloot,Boezemsloot,subclass


Onderstaande cell maakt van het resultaat van de bovenstaande query een json file die vervolgens door D3.js gebruikt kan worden..

In [29]:
G=nx.from_pandas_edgelist(q, 'parent_name','child_name' , edge_attr='type')
H = nx.convert_node_labels_to_integers(G,label_attribute='name')
    
# Output
data = json_graph.node_link_data(H)

with open('imbor_graph.json', 'w') as outfile:
    json.dump(data, outfile)

De ```%%html <div>``` instructie hieronder zet een 'canvas' klaar waarop de visualisatie getekend kan worden door de javascript functie in de cell daaronder...

In [30]:
%%html
<div id="d3-force-graph"></div>

In [31]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths:
    {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
  // The code in this block is executed when the
  // d3.js library has been loaded.

  // First, we specify the size of the canvas
  // containing the visualization (size of the
  // <div> element).
  var width = 800, height = 500;

  // We create a color scale.
  var color = d3.scale.category10();

  // We create a force-directed dynamic graph layout.
  var force = d3.layout.force()
    .charge(-120)
    .linkDistance(30)
    .size([width, height]);
    
    //setup function to drag the nodes apart and make them 'stick'
  var drag = force.drag()
    .on("dragstart", dragstart);

  // In the <div> element, we create a <svg> graphic
  // that will contain our interactive visualization.
  var svg = d3.select("#d3-force-graph").select("svg")
  if (svg.empty()) {
    svg = d3.select("#d3-force-graph").append("svg")
          .attr("width", width)
          .attr("height", height);
  }

  // We load the JSON file.
  d3.json("imbor_graph.json", function(error, graph) {
    // In this block, the file has been loaded
    // and the 'graph' object contains our graph.

    // We load the nodes and links in the
    // force-directed graph.
    force.nodes(graph.nodes)
      .links(graph.links)
      .start();

    // We create a <line> SVG element for each link
    // in the graph.
    var link = svg.selectAll(".link")
      .data(graph.links)
      .enter().append("line")
      .attr("class", "link") 
      .attr("stroke", "#999")
      .attr("stroke-opacity", 0.6);

    // We create a <circle> SVG element for each node
    // in the graph, and we specify a few attributes.
    var node = svg.selectAll(".node")
      .data(graph.nodes)
      .enter().append("circle")
      .attr("class", "node")
      .attr("r", 10)  // radius
      .style("fill", function(d) {
         // The node color depends on the club.
         return color(d.name);
      })
      .call(drag);
      
    // The name of each node is the node number.
    node.append("title")
        .text(function(d) { return d.name; });
      
    // Create a <text> SVG element for the labels
    var text = svg.selectAll(".text")
        .data(graph.nodes)
        .enter().append("text")
        .attr("dy", ".35em")
        .style("font-size",  "10px")
        .text(function(d) {
        return d.name;
      });
      

    // We bind the positions of the SVG elements
    // to the positions of the dynamic force-directed
    // graph, at each time step.
    force.on("tick", function() {
      link.attr("x1", function(d){return d.source.x})
          .attr("y1", function(d){return d.source.y})
          .attr("x2", function(d){return d.target.x})
          .attr("y2", function(d){return d.target.y});

     node.attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; });   
    text.attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; });
     });
     
  });
});

// function to drag the nodes apart
function dragstart(d) {
  d3.select(this).classed("fixed", d.fixed = true);
}

<IPython.core.display.Javascript object>